In [1]:
import json
import os
import markdown
from docx import Document
from bs4 import BeautifulSoup
from docx.shared import Pt
from docx.oxml.ns import qn
from tqdm import tqdm

In [2]:
import os
import json
from tqdm import tqdm
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
import markdown
from bs4 import BeautifulSoup

# 1. Word 문서 생성
doc = Document()

# 2. 폰트 설정 함수 정의 (한글 지원)
def set_font(run, font_name="맑은 고딕", size=11, bold=False):
    run.font.size = Pt(size)
    run.bold = bold
    run.font.name = font_name
    r = run._element
    r.rPr.rFonts.set(qn('w:eastAsia'), font_name)

# 3. JSON 파일들이 저장된 폴더 경로 설정
folder_path = "./artworks"  # JSON 파일 경로 수정
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

# 4. Markdown 형식의 콘텐츠를 Word 문서로 변환하는 함수
def markdown_to_docx(md_content, doc):
    html = markdown.markdown(md_content)
    soup = BeautifulSoup(html, "html.parser")

    for element in soup.descendants:
        if element.name == "p":
            para = doc.add_paragraph(element.get_text())
            set_font(para.runs[0])
        elif element.name in ["h1", "h2", "h3"]:
            heading_level = int(element.name[1])
            para = doc.add_heading(element.get_text(), level=heading_level)
            set_font(para.runs[0], bold=True)
        elif element.name == "li":
            para = doc.add_paragraph(element.get_text(), style='ListBullet')
            set_font(para.runs[0])

# 5. JSON 파일 반복 처리 및 데이터 모으기
all_data = []

for filename in tqdm(json_files, desc="Reading JSON files", unit="file", ncols=80):
    file_path = os.path.join(folder_path, filename)

    with open(file_path, 'r', encoding='utf-8') as f:
        data_list = json.load(f)
        all_data.extend(data_list)

# 6. 작품 번호(wrkMngNo) 기준 정렬
all_data = sorted(all_data, key=lambda x: int(x.get('rnum', 0)))

# 7. 정렬된 데이터를 Word 문서에 기록 (진행 퍼센트 표시)
for idx, data in enumerate(tqdm(all_data, desc="Writing to Word", unit="entry", ncols=80)):
    md_content = f"""
- 작품명: {data.get('wrkNm', 'N/A')} / {data.get('wrkNmCh', 'N/A')} / {data.get('wrkNmEng', 'N/A')}
- 작가: {data.get('artistnm', 'N/A')} / {data.get('artistnmEng', 'N/A')}
- 작품 번호: {data.get('rnum', 'N/A')}
- 제작 연도: {data.get('wrkProdTermEnd', 'N/A')}
- 크기: {data.get('detail_dimensions', 'N/A')}
- 재료: {data.get('detail_material', 'N/A')}
- 카테고리: {data.get('detail_category', 'N/A')}
- 작품 설명: {data.get('detail_desc', 'N/A')}
"""

    # Markdown을 Word 문서로 변환
    markdown_to_docx(md_content, doc)

    # 구분선 추가
    doc.add_paragraph()
    doc.add_paragraph()  # 빈 줄 추가

# 8. Word 문서 저장
output_path = "./dataset2.docx"
doc.save(output_path)

print(f"모든 JSON 데이터가 '{output_path}'에 작품 번호 순서대로 저장되었습니다!")


Writing to Word:   0%|                             | 0/11029 [00:00<?, ?entry/s]/home/chae/ch/lib/python3.12/site-packages/docx/styles/styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)
Writing to Word: 100%|█████████████████| 11029/11029 [04:13<00:00, 43.58entry/s]


모든 JSON 데이터가 './dataset2.docx'에 작품 번호 순서대로 저장되었습니다!


In [5]:
from docx import Document
from tqdm import tqdm

# 1. 원본 Word 문서 열기
input_path = "./dataset.docx"
doc = Document(input_path)

# 2. 문서의 모든 단락(paragraphs) 가져오기
paragraphs = doc.paragraphs
total_paragraphs = len(paragraphs)

# 3. 각 문서에 포함될 단락 수 계산
chunk_size = total_paragraphs // 5  # 나눌 단위 크기
remainder = total_paragraphs % 5  # 남은 단락 수

# 4. 5개의 문서를 생성하고 저장
start_idx = 0

for i in tqdm(range(5), desc="Creating split Word files", unit="file", ncols=80):
    # 남은 단락을 분배하기 위해 일부 문서에 +1 단락 추가
    end_idx = start_idx + chunk_size + (1 if i < remainder else 0)

    # 새로운 Word 문서 생성
    new_doc = Document()
    for para in paragraphs[start_idx:end_idx]:
        new_doc.add_paragraph(para.text)

    # 새로운 문서 저장 (파일명에 인덱스 번호 추가)
    output_path = f"./dataset_part_{i + 1}.docx"
    new_doc.save(output_path)

    # 다음 문서의 시작 인덱스 설정
    start_idx = end_idx

print("dataset.docx 파일이 5개로 나누어 저장되었습니다!")


Creating split Word files: 100%|████████████████| 5/5 [00:16<00:00,  3.26s/file]

dataset.docx 파일이 5개로 나누어 저장되었습니다!
